# $T_1$ under repeated measurement using QI 2.0

Let's first do the necessary imports and login in the Quantum Inspire website. 

In [ ]:
from qiskit import QuantumCircuit
from qiskit_quantuminspire.qi_provider import QIProvider
import matplotlib.pyplot as plt
import numpy as np
from utility_functions import *
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display

provider = QIProvider()
backend = "Starmon 7"

!qi login "https://staging.qi2.quantum-inspire.com"

Using the function new_day() will create a folder in the path specified in the utility_functions.py where we will save all our results.

In [ ]:
DataPath = new_day()

We can now create the circuit using Qiskit.

In [ ]:
nr_qubits = 7
nr_bits = 15

wait_time = 5-6 # s
IGate_duration = 20e-9 # s
nr_IG_wait_time = int(wait_time / IGate_duration)

qubit_index = 0

circuit = QuantumCircuit(nr_qubits, nr_bits) # define the quantum circuit
circuit.x(qubit_index)
for measurement_index in range(nr_bits - 1):
    circuit.measure(qubit = qubit_index, cbit = measurement_index)
    circuit.delay(nr_IG_wait_time, qubit_index)
circuit.measure(qubit = qubit_index, cbit = measurement_index + 1)

If you want to see the circuit run the cell below.

In [ ]:
circuit.draw('mpl')

Choose the name of the files that we will create and save. 

In [9]:
basename = 'Example'
suffix = 'T1_under_measurement_w10us'
histname, circuit_name, rawname = prepare_file(basename, suffix, doraw=1)

Specify the number of shots and run the algorithm.

In [10]:
nr_shots = 2**14

results = api_run_and_save(0, Qcircuit = circuit, rawdata_filename = rawname, circuit_name=circuit_name,
                           histname = histname, shots=nr_shots, backend_name=backend, get_results = True,
                           get_hist_data=True, measurement_list=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], get_raw_data=True)

### Data analysis

The data are saved in a csv file in the format:

In [ ]:
img = Image.open(r'C:\Users\rdicarlo\OneDrive - Delft University of Technology\Desktop\Delft\QI_2.0\img\raw_data_csv.png')
img = img.resize((200, 500))
display(img)

In [11]:
df = pd.read_csv(rawname+"_0.csv")
measurement_outcomes_list = np.zeros((len(df["Raw data values"]), (len(df["Raw data values"][0])-1)), dtype = np.int8)

for shot_index in range(len(df["Raw data values"])):
    for measurement_index in range(1, len(df["Raw data values"][0])):
        measurement_outcomes_list[shot_index][(measurement_index - 1)] = np.int8(df["Raw data values"][shot_index][len(df["Raw data values"][0]) - measurement_index])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 3), sharey = 'row', sharex = 'col')
ax.plot(np.arange(len(np.sum(measurement_outcomes_list, 0))) * (wait_time * 1e6 + 1), np.sum(measurement_outcomes_list, 0) / nr_shots, '-o')

ax.set_ylim(0, 1)
ax.set_xlim(-1.5, max(np.arange(len(np.sum(measurement_outcomes_list, 0))) * (wait_time * 1e6 + 1)) + 1.5)

ax.set_title('$T_1$ under repeated measurement')
ax.set_ylabel('Fraction')
ax.set_xlabel('Time (us)')